In [2]:
import ee
import folium
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# This is needed in python to add layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=WnMjX9Vf7iezqSBWbT-A7cFrZeJ_Lq6C1a6N23Bi7aw&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiKE4PeXKU_VX1Aiu_imk5MO1ohv85QLn4iZ91FBtc-CC9Du2xb728

Successfully saved authorization token.


In [3]:
# Define region around Milan
region = ee.Geometry.Rectangle(45.712377, 5.296382, 42.604597, 16.850474)
region.getInfo()['coordinates']

[[[42.604597, 5.296382],
  [45.712377, 5.296382],
  [45.712377, 16.850474],
  [42.604597, 16.850474],
  [42.604597, 5.296382]]]

import io
from PIL import Image as ioImage

In [23]:


# Define a map centered on Milan, Italy.
myLocation = [45.464203, 9.189982]
map_bare = folium.Map(location=myLocation, zoom_start=7)

dataset = ee.Image('CSP/ERGo/1_0/Global/SRTM_topoDiversity')
srtmTopographicDiversity = dataset.select('constant')
srtmTopographicDiversityVis = {
  'min': 0.0,
  'max': 1.0,
};

map_bare.add_ee_layer(srtmTopographicDiversity, srtmTopographicDiversityVis, 'SRTM Topographic Diversity')
map_bare.save('Milan_SRTM.html')
map_bare




In [17]:
# Creates folder in Google Drive but does not create image
task = ee.batch.Export.image.toDrive(**{
                                     'image' : dataset,    
                                     'description': 'MilanSRTM',
                                     'folder': 'HW4_Images2',
                                     'scale' : 30,
                                     'region' : region.getInfo()['coordinates']})

task.start()
task.status()

{'creation_timestamp_ms': 1647029702314,
 'description': 'MilanSRTM',
 'id': '7CLHA6S24ZOIOYFBFQ2G2MWA',
 'name': 'projects/earthengine-legacy/operations/7CLHA6S24ZOIOYFBFQ2G2MWA',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1647029702314}

In [21]:
from re import S
# Time series of Atmospheric conditions over Milan

startDate = "2018-04-01" # set start date
endDate = "2018-04-04"   # set end date

dataset2 = ee.ImageCollection('COPERNICUS/S3/OLCI').filterDate(startDate, endDate) # collect images
rgb = dataset2.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance']).median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198])) # define coloration

# Oa08_radiance	W m^-2 sr^-1 μm^-1	0.00876539	665nm/10nm	
#       Chl (2^nd Chl abs. max.), sediment, yellow substance/vegetation

# Oa06_radiance	W m^-2 sr^-1 μm^-1	0.0123538	560nm/10nm	
#       Chlorophyll reference (Chl minimum)

# Oa04_radiance	W m^-2 sr^-1 μm^-1	0.0115198	490nm/10nm	
#       High Chl, other pigments


visParams = {'min': 0, 'max': 12, 'gamma': 1.5} # define visualization parameters

# add layers to maps
map_zones_atmos = folium.Map(location=myLocation, zoom_start=8) 
map_zones_atmos.add_ee_layer(rgb, visParams, 'RGB')
map_zones_atmos.save('Milan_Atmos.html')

map_zones_atmos



In [ ]:
# Just for Fun
# Load an ice coverage image onto Iceland.

dataset = ee.FeatureCollection('GLIMS/current')

visParams = {'palette': ['blue', 'cyan', 'green'], 'min': 0, 'max': 10, 'opacity': 0.5}

image = ee.Image().float().paint(dataset, 'area')

map_zones_ice = folium.Map(location=myLocation, zoom_start=7)
map_zones_ice.add_ee_layer(image, visParams, 'GLIMS/current')
map_zones_ice